In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cdist
from scipy.stats import mode
from sklearn.model_selection import train_test_split

# KNN
In this exercise we will implement the KNN algorithm. We will start with some data, generated using the `make_blobs` function from scikit learn. Read the documentation to learn how to influence the dataset generation procedure. 

Next, we will save a portion of the data for testing using scikit learn's `train_test_split` function. 

In [ ]:
X,y,centers = make_blobs(
    n_samples=1000,
    n_features=2,
    return_centers=True,
)

# TODO: Shuffle and split the data


# Plot the training data
plt.scatter(X_train[:,0], X_train[:,1], c=y_train)

plt.scatter(centers[:,0], centers[:,1], c='r', s=120)
plt.legend(["data", "centers"])
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$")

Next, let us create a KNN class. We can then implement the following:
### Constructor
`__init__` method, which will take as argument `K` (the number of neighbors to use) and store it as a property of the class.
### Fit
Fit method should simply store the training data X and y as properties.
### Predict
Predict function should take the input data and return the predicted labels.
Moreover, we would like to:
1. Check if the model has been already fitted
2. Calculate the squared euclidean distance between each point in X and each point in obj.X
3. Find the K nearest neighbors for each point in X, and return the most common label among them


## Hints
Have a look at the `cdist` function from `scipy.spatial.distance`. 

In [ ]:
class KNN:
    def __init__(self, K):
        ...

    def fit(self,X, y):
        ...

    def predict(self,X):
        ...

Now, we can instantiate a `KNN` object and fit it to the training data. Then, predict the labels of the test data and plot the results.

In [ ]:
# TODO: Instantiate, fit and predict

plt.scatter(X_test[:,0], X_test[:,1], c=y_test, s=120, edgecolors='k', alpha=0.6)
plt.scatter(X_test[:,0], X_test[:,1], c=preds, edgecolors='k', alpha=.6)

## Decision boundary
In order to investigate where the decision boundary is, we can create a `meshgrid` of points in the two dimensional input space. Then, we can simply call the `predict` method for each point. The output can be reshaped to a a grid and plotted using `countourf`.

In [ ]:
res = 50
grid_x = np.linspace(X[:,0].min(), X[:,0].max(), res)
grid_y = np.linspace(X[:,1].min(), X[:,1].max(), res)
xx,yy = np.meshgrid(grid_x,grid_y)

# TODO: complete